Here we will build an convolutional neural network to predict the CIFAR-10 data.
The script provided will download and unzip the CIFAR-10 data.  Then it will start training a CNN from scratch.  You should see similar output at the end to the following two graphs.

![Loss and Accuracy](https://github.com/nfmcclure/tensorflow_cookbook/blob/master/08_Convolutional_Neural_Networks/images/03_cnn2_loss_acc.png?raw=true)

Here we see the training loss (left) and the test batch accuracy (right).


In [16]:
import os
import sys
import tarfile
import tensorflow as tf
import numpy as np
from tensorflow.python.framework import ops
import matplotlib.pyplot as plt
from six.moves import urllib
ops.reset_default_graph()

#change dir
abspath = os.path.abspath(__name__)
dirname = os.path.dirname(abspath)
os.chdir(dirname)
# Start a graph session
sess = tf.Session()

# Set model parameters
batch_size = 128
data_dir = 'cifar10'
output_every = 50
generations = 20000
eval_every = 500
image_height = 32
image_width = 32
crop_height = 24
crop_width = 24
num_channels = 3
num_targets = 10
extract_folder = 'cifar-10-batches-bin'

cifar10_url = 'http://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz'

# Exponential Learning Rate Decay Params
learning_rate = 0.1
lr_decay = 0.1
num_gens_to_wait = 250.

# Extract model parameters
image_vec_length = image_height * image_width * num_channels
record_length = 1 + image_vec_length  # ( + 1 for the 0-9 label)

In [17]:
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    
# Check if file exists, otherwise download it
data_file = os.path.join(data_dir,'cifar-10-binary.tar.gz')
if os.path.isfile(data_file):
    pass
else:
    # Download file
    def progress(block_num,block_size,total_size):
        progress_info = [cifar10_url,float(block_num * block_size) / float(total_size) * 100.0]
        print('\r Downloading {} - {:.2f}%'.format(*progress_info), end="")
    filepath,_ = urllib.request.urlretrieve(cifar10_url,data_file,progress)
    # Extract file
    tarfile.open(filepath,'r:gz').extractall(data_dir)

In [ ]:
# Define CIFAR reader
'''
tf.FixedLengthRecordReader是读取固定长度字节数信息(针对bin文件使用FixedLengthRecordReader读取比较合适)，
结果表明下次调用时会接着上次读取的位置继续读取文件，而不会从头开始读取。
'''
def read_cifar_files(filename_queue,distort_image=True):
    reader = tf.FixedLengthRecordReader(record_bytes=record_length)
    